In [1]:
import json
import pandas as pd
from pathlib import Path
from pprint import pprint
import datetime
import time
import numpy as np
import matplotlib as plt

import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

%matplotlib inline

In [2]:
#path_results = Path('/media/discoD/models/elmo/ner/results_20_epochs_ibm')
#path_results = Path('/media/discoD/models/elmo/ner/results_1_epoch_harem_all_combinations')
path_results = Path('/opt/models/elmo/ner')

In [3]:
for folder in path_results.iterdir():
    print(folder)

/opt/models/elmo/ner/metrics_glove_5b-jur2vec-nilc_5ª.json
/opt/models/elmo/ner/results_datalawyer_jur_embeddings_sem_lower_ner_sem_lower
/opt/models/elmo/ner/metrics_glove_5b-jur2vec-nilc_4ª.json
/opt/models/elmo/ner/results_datalawyer
/opt/models/elmo/ner/results_old
/opt/models/elmo/ner/metrics_glove_5b-jur2vec-nilc_1ª.json
/opt/models/elmo/ner/results_datalawyer_all_reps.tar.gz
/opt/models/elmo/ner/model.tar.gz
/opt/models/elmo/ner/results_none
/opt/models/elmo/ner/metrics_glove_5b-nilc_5ª.json
/opt/models/elmo/ner/results
/opt/models/elmo/ner/metrics_glove_5b-jur2vec-nilc-2_2ª.json
/opt/models/elmo/ner/results_20_epochs.tar.gz
/opt/models/elmo/ner/results_datalawyer-ft
/opt/models/elmo/ner/results_20e
/opt/models/elmo/ner/metrics_glove-lc-5bjur2vecnilc2.json
/opt/models/elmo/ner/results (harem total, pre limpeza de cache do allennlp)
/opt/models/elmo/ner/metrics_glove_5b-jur2vec-nilc-2_1ª.json
/opt/models/elmo/ner/results_datalawyer_all_reps_sem_jur.tar.gz
/opt/models/elmo/ner/res

In [4]:
metrics = dict()
for results_file in path_results.iterdir():
    if results_file.name.startswith('metrics_glove_5b'):
        #print('Parsing data from %s' % results_file.name)
        metrics[results_file.name] = json.loads(results_file.read_bytes())
print(len(metrics))

15


In [5]:
def get_seconds(time_str):
    x = time.strptime(time_str.split('.')[0],'%H:%M:%S')
    return datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
def get_average_epoch_duration(metrics):
    seconds = get_seconds(metrics['training_duration'])
    training_epochs = metrics['training_epochs'] + 1
    return seconds / training_epochs

In [6]:
TRAINING_ID = 'Training_ID'
SCENARIO = 'Scenario'
MODEL = 'Model'
REPRESENTATION = 'Representation'
EMBEDDING = 'Embedding'
EMBEDDING_TYPE = 'Embedding_Type'
BATCH_SIZE = 'Batch_Size'
ELMO_DROPOUT = 'ELMo_Dropout'
TRAINING_NUMBER = 'Training_Number'
EXECUTION_NUMBER = 'Execution_Number'
BEST_EPOCH = 'Best_Epoch'
TRAINING_EPOCHS = 'Training_Epochs'
TRAINING_DURATION = 'Training_Duration'
TOTAL_DURATION = 'Total_Duration(s)'
AVERAGE_EPOCH_DURATION = 'Average_Epoch_Duration(s)'
TRAINING_ACCURACY = 'Training_Accuracy'
TRAINING_ACCURACY_TOP3 = 'Training_Accuracy_Top-3'
TRAINING_PRECISION = 'Training_Precision'
TRAINING_RECALL = 'Training_Recall'
TRAINING_F1_MEASURE = 'Training_F1-Measure'
TRAINING_LOSS = 'Training_Loss'
BEST_VALIDATION_ACCURACY = 'Best_Validation_Accuracy'
BEST_VALIDATION_ACCURACY_TOP3 = 'Best_Validation_Accuracy_Top-3'
BEST_VALIDATION_PRECISION = 'Best_Validation_Precision'
BEST_VALIDATION_RECALL = 'Best_Validation_Recall'
BEST_VALIDATION_F1_MEASURE = 'Best_Validation_F1-Measure'
BEST_VALIDATION_LOSS = 'Best_Validation_Loss'
TEST_ACCURACY = 'Test_Accuracy'
TEST_ACCURACY_TOP3 = 'Test_Accuracy_Top-3'
TEST_PRECISION = 'Test_Precision'
TEST_RECALL = 'Test_Recall'
TEST_F1_MEASURE = 'Test_F1_Measure'
TEST_LOSS = 'Test_Loss'
columns = [TRAINING_ID, SCENARIO, MODEL, REPRESENTATION, EMBEDDING, EMBEDDING_TYPE, TRAINING_NUMBER, EXECUTION_NUMBER, BEST_EPOCH, TRAINING_EPOCHS, TRAINING_DURATION, TOTAL_DURATION, AVERAGE_EPOCH_DURATION, TRAINING_ACCURACY, TRAINING_ACCURACY_TOP3, TRAINING_PRECISION, TRAINING_RECALL, TRAINING_F1_MEASURE, TRAINING_LOSS, BEST_VALIDATION_ACCURACY, BEST_VALIDATION_ACCURACY_TOP3, BEST_VALIDATION_PRECISION, BEST_VALIDATION_RECALL, BEST_VALIDATION_F1_MEASURE, BEST_VALIDATION_LOSS, TEST_ACCURACY, TEST_ACCURACY_TOP3, TEST_PRECISION, TEST_RECALL, TEST_F1_MEASURE, TEST_LOSS]
columns

['Training_ID',
 'Scenario',
 'Model',
 'Representation',
 'Embedding',
 'Embedding_Type',
 'Training_Number',
 'Execution_Number',
 'Best_Epoch',
 'Training_Epochs',
 'Training_Duration',
 'Total_Duration(s)',
 'Average_Epoch_Duration(s)',
 'Training_Accuracy',
 'Training_Accuracy_Top-3',
 'Training_Precision',
 'Training_Recall',
 'Training_F1-Measure',
 'Training_Loss',
 'Best_Validation_Accuracy',
 'Best_Validation_Accuracy_Top-3',
 'Best_Validation_Precision',
 'Best_Validation_Recall',
 'Best_Validation_F1-Measure',
 'Best_Validation_Loss',
 'Test_Accuracy',
 'Test_Accuracy_Top-3',
 'Test_Precision',
 'Test_Recall',
 'Test_F1_Measure',
 'Test_Loss']

In [7]:
def get_training_data_from_id(training_id):
    data = training_id.split('_')
    print(data)
    if len(data) == 4:
        return {EMBEDDING: data[1], MODEL: data[2], EXECUTION_NUMBER: data[3]}
    else:
        print('Check id: %s' % training_id)

In [8]:
training_data = []
for training_id, metric in metrics.items():
    training_metrics = metrics[training_id]
    data = get_training_data_from_id(training_id)
    data[TRAINING_ID] = training_id
    data[BEST_EPOCH] = training_metrics['best_epoch'] if training_metrics else None
    data[TRAINING_EPOCHS] = training_metrics['training_epochs'] + 1 if training_metrics else None
    data[TRAINING_DURATION] = training_metrics['training_duration'] if training_metrics else None
    data[TOTAL_DURATION] = get_seconds(training_metrics['training_duration']) if training_metrics else None
    data[AVERAGE_EPOCH_DURATION] = get_average_epoch_duration(training_metrics) if training_metrics else None
    data[TRAINING_ACCURACY] = training_metrics['training_accuracy'] if training_metrics else None
    data[TRAINING_ACCURACY_TOP3] = training_metrics['training_accuracy3'] if training_metrics else None
    data[TRAINING_PRECISION] = training_metrics['training_precision-overall'] if training_metrics else None
    data[TRAINING_RECALL] = training_metrics['training_recall-overall'] if training_metrics else None
    data[TRAINING_F1_MEASURE] = training_metrics['training_f1-measure-overall'] if training_metrics else None
    data[TRAINING_LOSS] = training_metrics['training_loss'] if training_metrics else None
    data[BEST_VALIDATION_ACCURACY] = training_metrics['best_validation_accuracy'] if training_metrics else None
    data[BEST_VALIDATION_ACCURACY_TOP3] = training_metrics['best_validation_accuracy3'] if training_metrics else None
    data[BEST_VALIDATION_PRECISION] = training_metrics['best_validation_precision-overall'] if training_metrics else None
    data[BEST_VALIDATION_RECALL] = training_metrics['best_validation_recall-overall'] if training_metrics else None
    data[BEST_VALIDATION_F1_MEASURE] = training_metrics['best_validation_f1-measure-overall'] if training_metrics else None
    data[BEST_VALIDATION_LOSS] = training_metrics['best_validation_loss'] if training_metrics else None
    data[TEST_ACCURACY] = training_metrics['test_accuracy'] if training_metrics else None
    data[TEST_ACCURACY_TOP3] = training_metrics['test_accuracy3'] if training_metrics else None
    data[TEST_PRECISION] = training_metrics['test_precision-overall'] if training_metrics else None
    data[TEST_RECALL] = training_metrics['test_recall-overall'] if training_metrics else None
    data[TEST_F1_MEASURE] = training_metrics['test_f1-measure-overall'] if training_metrics else None
    data[TEST_LOSS] = training_metrics['test_loss'] if training_metrics else None
    training_data.append(data)

['metrics', 'glove', '5b-jur2vec-nilc', '5ª.json']
['metrics', 'glove', '5b-jur2vec-nilc', '4ª.json']
['metrics', 'glove', '5b-jur2vec-nilc', '1ª.json']
['metrics', 'glove', '5b-nilc', '5ª.json']
['metrics', 'glove', '5b-jur2vec-nilc-2', '2ª.json']
['metrics', 'glove', '5b-jur2vec-nilc-2', '1ª.json']
['metrics', 'glove', '5b-nilc', '1ª.json']
['metrics', 'glove', '5b-nilc', '2ª.json']
['metrics', 'glove', '5b-nilc', '3ª.json']
['metrics', 'glove', '5b-jur2vec-nilc', '2ª.json']
['metrics', 'glove', '5b-jur2vec-nilc-2', '3ª.json']
['metrics', 'glove', '5b-nilc', '4ª.json']
['metrics', 'glove', '5b-jur2vec-nilc-2', '4ª.json']
['metrics', 'glove', '5b-jur2vec-nilc-2', '5ª.json']
['metrics', 'glove', '5b-jur2vec-nilc', '3ª.json']


In [9]:
pd.set_option('display.max_columns', 1000)
training_data_df = pd.DataFrame(training_data, columns=columns)
training_data_df 

,Training_ID,Scenario,Model,Representation,Embedding,Embedding_Type,Training_Number,Execution_Number,Best_Epoch,Training_Epochs,Training_Duration,Total_Duration(s),Average_Epoch_Duration(s),Training_Accuracy,Training_Accuracy_Top-3,Training_Precision,Training_Recall,Training_F1-Measure,Training_Loss,Best_Validation_Accuracy,Best_Validation_Accuracy_Top-3,Best_Validation_Precision,Best_Validation_Recall,Best_Validation_F1-Measure,Best_Validation_Loss,Test_Accuracy,Test_Accuracy_Top-3,Test_Precision,Test_Recall,Test_F1_Measure,Test_Loss
0,metrics_glove_5b-jur2vec-nilc_5ª.json,NaN,5b-jur2vec-nilc,NaN,glove,NaN,NaN,5ª.json,17,20,0:20:58.652226,1258.0,62.90,0.991271,0.991728,0.904055,0.895347,0.899680,28.379793,0.984149,0.984593,0.838545,0.868223,0.853126,59.118471,0.982485,0.983172,0.809271,0.834639,0.821759,71.357120
1,metrics_glove_5b-jur2vec-nilc_4ª.json,NaN,5b-jur2vec-nilc,NaN,glove,NaN,NaN,4ª.json,17,20,0:21:17.179115,1277.0,63.85,0.991136,0.991601,0.902709,0.892245,0.897446,28.520987,0.983724,0.984075,0.831769,0.867470,0.849244,60.894604,0.982028,0.982750,0.805576,0.837774,0.821360,73.878596
2,metrics_glove_5b-jur2vec-nilc_1ª.json,NaN,5b-jur2vec-nilc,NaN,glove,NaN,NaN,1ª.json,16,20,0:21:10.487172,1270.0,63.50,0.991424,0.991866,0.901615,0.893224,0.897400,28.465786,0.982725,0.983299,0.846211,0.857681,0.851907,58.546244,0.981852,0.982697,0.810146,0.826019,0.818005,74.598960
3,metrics_glove_5b-nilc_5ª.json,NaN,5b-nilc,NaN,glove,NaN,NaN,5ª.json,17,20,0:20:50.753509,1250.0,62.50,0.990921,0.991336,0.902881,0.895510,0.899180,29.555327,0.985943,0.986332,0.849668,0.868223,0.858845,52.392261,0.982644,0.983278,0.817764,0.836991,0.827266,73.655577
4,metrics_glove_5b-jur2vec-nilc-2_2ª.json,NaN,5b-jur2vec-nilc-2,NaN,glove,NaN,NaN,2ª.json,16,20,0:21:28.658862,1288.0,64.40,0.990913,0.991359,0.897707,0.888327,0.892992,30.266560,0.984112,0.984649,0.857791,0.853916,0.855849,56.281653,0.982397,0.983154,0.829554,0.831505,0.830528,74.734740
5,metrics_glove_5b-jur2vec-nilc-2_1ª.json,NaN,5b-jur2vec-nilc-2,NaN,glove,NaN,NaN,1ª.json,17,20,0:21:29.090507,1289.0,64.45,0.991021,0.991466,0.900563,0.888653,0.894568,30.412279,0.985111,0.985629,0.858327,0.857681,0.858004,53.939067,0.983489,0.984193,0.831783,0.840909,0.836321,70.001235
6,metrics_glove_5b-nilc_1ª.json,NaN,5b-nilc,NaN,glove,NaN,NaN,1ª.json,17,20,0:21:13.258287,1273.0,63.65,0.990875,0.991274,0.898734,0.892571,0.895642,29.280662,0.986036,0.986406,0.852616,0.871235,0.861825,53.834427,0.982345,0.982961,0.817625,0.836207,0.826811,75.613903
7,metrics_glove_5b-nilc_2ª.json,NaN,5b-nilc,NaN,glove,NaN,NaN,2ª.json,18,20,0:22:26.221185,1346.0,67.30,0.990994,0.991390,0.901068,0.895184,0.898116,29.238075,0.983853,0.984205,0.860643,0.846386,0.853455,57.045874,0.983137,0.983770,0.839017,0.829154,0.834056,71.311208
8,metrics_glove_5b-nilc_3ª.json,NaN,5b-nilc,NaN,glove,NaN,NaN,3ª.json,17,20,0:21:27.818027,1287.0,64.35,0.990963,0.991363,0.903640,0.895673,0.899639,29.062878,0.986313,0.986813,0.849335,0.865964,0.857569,54.169824,0.982556,0.983242,0.814220,0.834639,0.824303,73.730213
9,metrics_glove_5b-jur2vec-nilc_2ª.json,NaN,5b-jur2vec-nilc,NaN,glove,NaN,NaN,2ª.json,16,20,0:21:45.598659,1305.0,65.25,0.991451,0.991904,0.905533,0.895184,0.900328,28.781158,0.984334,0.984871,0.846553,0.859940,0.853194,57.753781,0.983137,0.983929,0.816577,0.833856,0.825126,73.066393


In [10]:
str(datetime.timedelta(seconds=training_data_df[TOTAL_DURATION].mean()))

'0:21:25.066667'

In [11]:
default_exported_columns = ['mean', 'min', 'max', 'std']
renamed_columns = ['Contagem', 'F-Score', 'Desvio Padrão', 'Mínimo', '25%', '50%', '75%', 'Máximo']
exported_columns_names = ['F-Score', 'Mínimo', 'Máximo', 'Desvio Padrão']
scenario_map = {'selective': 'Seletivo', 'total': 'Total'}
model_map = {'harem-ft': 'Sim', 'harem': 'Não'}
representation_map = {'ELMo+CNN+Embeddings': 'ELMo+CNN+Vetor', 'ELMo+Embeddings': 'ELMo+Vetor'}
embedding_type_map = {'skip': 'Skip-Gram', 'No': 'Sem Vetor', 'cbow': 'CBoW'}
embedding_map = {'wang2vec': 'Wang2Vec', 'glove': 'GloVe', 'word2vec': 'Word2Vec', 'No': 'Sem Vetor', 'fasttext': 'FastText'}
replacements_map = {**scenario_map, **model_map, **representation_map, **embedding_type_map, **embedding_map}

def get_group_csv(index_names, group_name, group_columns, target_value, index_map, dataframe=training_data_df, exported_columns=default_exported_columns):
    if type(index_names) == str:
        index_names = [index_names]
    group = dataframe.groupby(group_columns).describe()[target_value].sort_values(by='mean', ascending=False)
    for column in exported_columns:
        group[column] = group[column] * 100
    group.index.names = index_names
    group = group.rename(index=index_map)
    group.columns = renamed_columns
    #group.to_csv('grupo_' + group_name + '_75_epochs_mestrado.csv', columns=exported_columns_names, float_format = '%.2f%%')
    return group

In [13]:
training_data_df.groupby([MODEL]).describe()[TEST_F1_MEASURE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Model,,,,,,,,
5b-jur2vec-nilc-2,5.0,0.832945,0.002848,0.830528,0.830793,0.831321,0.835762,0.836321
5b-nilc,5.0,0.827701,0.003727,0.824303,0.826070,0.826811,0.827266,0.834056
5b-jur2vec-nilc,5.0,0.822310,0.003025,0.818005,0.821360,0.821759,0.825126,0.825299


In [ ]:
training_data_df.groupby([EMBEDDING, EMBEDDING_TYPE]).describe()[TEST_F1_MEASURE].sort_values(by='mean', ascending=False)

In [ ]:
training_data_df[BEST_EPOCH].hist()

In [ ]:
training_data_df[TOTAL_DURATION].describe()

In [ ]:
training_data_df.groupby([TRAINING_NUMBER]).describe()[TEST_F1_MEASURE].sort_values(by='mean', ascending=False)

In [ ]:
rp.summary_cont(training_data_df[TEST_F1_MEASURE])

In [ ]:
rp.summary_cont(training_data_df.groupby(EMBEDDING))[TEST_F1_MEASURE]

In [ ]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

def print_anova(parameter):
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('Test_F1_Measure ~ C({0})'.format(parameter), training_data_df).fit()
    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .20f}")
    print(model.summary())
    res = sm.stats.anova_lm(model, typ= 2)
    print(res)
    print(anova_table(res))
    mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[TEST_F1_MEASURE], training_data_df[parameter])
    mc_results = mc.tukeyhsd()
    print(mc_results)

In [ ]:
print_anova(EMBEDDING)

In [ ]:
training_data_df[(training_data_df.Embedding == 'wang2vec') & (training_data_df.Embedding_Type == 'skip') & (training_data_df.Representation == 'ELMo+Embeddings')]